### 🌐 Step 0: Install required imports

In [ ]:
# Pip installations
%pip install usd-core --quiet
%pip install spark --quiet
%pip install thefuzz --quiet
# Import Open USD MSFabric Package
%pip install /lakehouse/default/Files/openusd_msfabric_toolkit-0.1.0-py3-none-any.whl --quiet
# Imports
from pxr import Usd, UsdGeom, Sdf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from openusd_msfabric_toolkit import OpenUSDToolkit
# Create Spark Session
spark = SparkSession.builder.getOrCreate()

### 📝 Step 1: Provide name of USD file, Asset data table, and Lakehouse
Please update the Python cell below.
Please note that your files must be located in the `/lakehouse/default/Files/` directory.

Note, please just supply the file name - not the file path. Example file name:
```
PCR_8FT2_ALL_Complete_wOneRobot.usd
```

In [ ]:
# USD File Details
usd_file_name = "iss.usdc"  # Name of the USD file you want to enrich
usd_lakehouse_name = "yourlakehouse" # Name of the lakehouse in which your USD file is stored

# Asset Data Details
entity_name = "Module" # Name of the Entity Type you wish to match with the USD Metadata. This is case sensitive!
dtb_item = "yourdtb" # Name of your DTB Instance. This is case sensitive!

# Do not modify
usd_file_path = f"/lakehouse/default/Files/{usd_file_name}"

### 📂 Step 2: Extract metadata from your USD file
This step extracts metadata from prims of type Xform. This data will be stored in a table titled **ExtractedUSDMetadata** in your **default** Lakehouse.

In [ ]:
# Do not modify
df_metadata = OpenUSDToolkit.read_usd_metadata(usd_file_path, spark)
sql_query = f"SELECT * FROM {usd_lakehouse_name}.extractedusdmetadata"
df_usd_metadata = spark.sql(sql_query)
display(df_usd_metadata)

### 🧩 Step 3: Relate Asset Data and extracted USD metadata
This step fuzzy matches your asset data in your Lakehouse with the extracted USD metadata.

In [ ]:
# Do not modify
df_entity_instances = OpenUSDToolkit.read_entity_instances(spark, dtb_item, entity_name)
display(df_entity_instances)

The fuzzy matching threshold determines how closely the asset names must match the USD metadata in order to be considered a valid match. You may wish to set the thresholds of matching strength with the variable `fuzzy_threshold` . We are using the Fuzzy matching library called [TheFuzz](https://github.com/seatgeek/thefuzz) and setting these thresholds may produce different results.



```
Range: 0 to 100
Higher values (e.g., 90–100) = stricter matching
Lower values (e.g., 60–80) = more flexible, but may include incorrect matches
If no threshold is provided, a default of 80 will be used.
```

If you're unsure, try starting with a value like 85, then adjust based on match quality.

Setting the entity_instance_col Will change the column to match against, however this can be left as default for we are using the name from the DTB Data Model.



In [ ]:
# Optional: Set a custom fuzzy match threshold (0–100)
# fuzzy_threshold = 50  # <-- Change this if needed

# Specify the column in your asset table that identifies your entity
entity_instance_col = "EntityInstanceDisplayId" # <-- Change this if needed

# Do not modify 
try:
    fuzzy_threshold
except NameError:
    fuzzy_threshold = 50  # Default threshold
result_df = OpenUSDToolkit.fuzzy_match_usd_assets(spark, df_usd_metadata, df_entity_instances, entity_instance_col, fuzzy_threshold)
display(result_df)

### 🧇 Step 4: Extract Matches to Table
- This will save the data matched above into a table we can use within DTB.
- You can optionally provide a `table_name` to store this output data.

In [ ]:
table_name = "MatchedUSDMetadata"
result_df.write.mode("overwrite").format("delta").saveAsTable(table_name)

### 🛠️ Step 5: Enrich Your USD File
- You can optionally provide a file name for the enriched USD file that will include additional properties based on the matched asset data.
- If you provide a new file name, the enriched USD data will be saved there.
- If you do not provide a new file name, "\_enriched_" will be appended to the original file path.

In [ ]:
# Optional: Provide a file name for the enriched USD file
enriched_usd_file_name = None  # e.g., "OneRobotEnriched.usd" or leave as None

# Do not modify
if enriched_usd_file_name:
    enriched_usd_file_path = f"/lakehouse/default/Files/{enriched_usd_file_name}"
else:
    import os
    base, ext = os.path.splitext(usd_file_name)
    enriched_usd_file_path = f"/lakehouse/default/Files/{base}_enriched{ext}"

OpenUSDToolkit.enrich_usd_with_dtb_assets(usd_file_path, enriched_usd_file_path, df_usd_metadata, result_df)
print(f"✅ USD file enriched successfully! File saved to: {enriched_usd_file_path}")
OpenUSDToolkit.print_usd_file_details(f"/lakehouse/default/Files/{base}_enriched{ext}", onlyDTBID=True)